In [ ]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch,RunnableLambda,RunnablePassthrough,RunnableParallel
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field
from typing import Literal

load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id='meta-llama/Llama-3.3-70B-Instruct',
    task='text-generation'
)

model = ChatHuggingFace(llm=llm)
parser = StrOutputParser()

In [10]:
class Feedback(BaseModel):
    sentiment: Literal['positive','negative'] = Field("Give the Sentiment of the feedback")
    
parser2 = PydanticOutputParser(pydantic_object=Feedback)

In [11]:
prompt1=PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_ins}',
    input_variables=['feedback'],
    partial_variables={'format_ins':parser2.get_format_instructions()}
)

In [12]:
prompt2=PromptTemplate(
    template="Write an appropiate response to this positive Feedback \n {feedback}",
    input_variables=["feedback"]
)

prompt3=PromptTemplate(
    template="Write an appropiate response to this negative Feedback \n {feedback}",
    input_variables=["feedback"]
)

In [13]:
combined = RunnableParallel({
    'sentiment' : prompt1 | model | parser2,
    'feedback' : RunnablePassthrough()
})

branch_chain = RunnableBranch(
    (lambda x: x["sentiment"].sentiment == "positive", prompt2 | model | parser),
    (lambda x: x["sentiment"].sentiment == "negative", prompt3 | model | parser),
    RunnableLambda(lambda x: "Could not determine sentiment.")
)

In [14]:
chain = combined | branch_chain

In [15]:
print(chain.invoke({'feedback': 'This is the best smartphone'}))

Here's a potential response to the positive feedback:

"Thank you so much for your kind words! We're thrilled to hear that you're enjoying your smartphone and that it's meeting your expectations. We're constantly working to improve and innovate, and feedback like yours makes it all worth it. If you have any other questions or need any assistance, don't hesitate to reach out. Thanks again for choosing our product!"


In [8]:
chain.get_graph().print_ascii()

        +-----------------------------------+      
        | Parallel<sentiment,feedback>Input |      
        +-----------------------------------+      
                  ***            ***               
                **                  ***            
              **                       **          
   +----------------+                    **        
   | PromptTemplate |                     *        
   +----------------+                     *        
            *                             *        
            *                             *        
            *                             *        
  +-----------------+                     *        
  | ChatHuggingFace |                     *        
  +-----------------+                     *        
            *                             *        
            *                             *        
            *                             *        
+----------------------+          +-------------+  
| PydanticOu